In [1]:
from databases import BDS,Crm,BDSMem,PosicaoDm1, Boletador,Bawm
from datetime import date, datetime, timedelta
from dateutil.relativedelta import relativedelta
from funcoes_datas import FuncoesDatas
import pandas as pd
from emailer import Email, EmailLer
from pretty_html_table import build_table
import numpy as np

[base_dados.py directory]: C:\Temp\Foundation
[databases.py directory]: C:\Temp\Foundation


In [2]:
##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
crm_fundos = Crm(load_cadfundos=True)
bds = BDS()
fdt = FuncoesDatas()
bol = Boletador()
posicao = PosicaoDm1
bawm = Bawm()

#retorna posição de todas as carteiras + fundos
posicao = PosicaoDm1()

#Retorna a data de hoje
hoje = bds.banco.hoje()

#Retorna as informações dos fundos
fundos = crm_fundos.fundo_cadastro()

#Retorna todos os produtos da casa
#produtos= crm.product()

#Retorna a data 1 do mes atual
hoje = bds.banco.hoje()
dia = hoje + relativedelta(months=-1)
mes,ano = hoje.month,hoje.year

#Traz o primeiro dia util do mes anterior
data_inicial_bol = datetime(ano,mes,1)




In [3]:
#Controle do Preço Médio

#Retorna o dia anterior
#dia = fdt.workday(hoje, n_dias=-1)
dia = datetime(2024,10,24)
boletas = bol.boletas_importadas_cockpit(dia)

#Retorna o preço médio por cliente
preco_medio=[]
for idx, row in boletas.iterrows():
    cliente=row['ContaCRMGuid']
    cliente_nome=row['ContaCRM']
    ativo = row['AtivoGuid']
    nome_ativo = row['AtivoNome']
    qtd = row['Quantidade']
    preco = row['Preco']
    corretora = row['Contraparte']
    pond_qtd = qtd / boletas[(boletas['ContaCRMGuid']==cliente)&(boletas['AtivoGuid']==ativo)][['Quantidade']].sum()[0]
    preco_medio_cliente = preco*pond_qtd
    preco_medio.append([cliente,cliente_nome,ativo,nome_ativo,qtd,preco,preco_medio_cliente,corretora])    
preco_medio_cliente = pd.DataFrame(preco_medio)    
preco_medio_cliente = preco_medio_cliente .groupby([0,1,2,3,7]).sum().reset_index()[[1,2,3,6,7]].sort_values(by=[6])  

#Retorna o preço médio por ativ

preco_medio_ativo= []
for idx, row in boletas.iterrows():
    ativo = row['AtivoGuid']
    nome_ativo = row['AtivoNome']
    qtd = row['Quantidade']
    preco = row['Preco']
    contraparte = row['Contraparte']
    pond_qtd = qtd / boletas[(boletas['Contraparte']==contraparte)&(boletas['AtivoGuid']==ativo)][['Quantidade']].sum()[0]
    preco_medio = preco*pond_qtd
    preco_medio_ativo.append([contraparte,ativo,preco_medio])    
preco_medio_ativo = pd.DataFrame(preco_medio_ativo)    
preco_medio_ativo = preco_medio_ativo.groupby([0,1]).sum().reset_index()

#Verificando preço médio
verificacao =pd.merge(left=preco_medio_cliente,right=preco_medio_ativo,left_on=[2,7],right_on=[1,0],how='left')[['1_x',3,6,'2_y',7]]

#Verificar os clientes que possuem veto
veto = bol.ordem_veto_power()[['IdOrdem']]
veto_dpm = bol.ordem_veto_dpm()[['IdOrdem']]
vetos = pd.concat([veto,veto_dpm])
vetos['Veto']='Possui_Veto'
vetos = pd.merge(left=boletas, right=vetos,on='IdOrdem',how='left')[['ContaCRM','Veto']].drop_duplicates()

#Unir todas as informações e identificar os problemas
verificacao.columns=['ContaCRM','Ativo','Preço_Exec','Preço_Médio_Dia','Corretora']
verificacao = pd.merge(left=verificacao, right=vetos,on='ContaCRM',how='left')
verificacao['Preço_Médio_Dia'] = verificacao['Preço_Médio_Dia'].astype('float')
verificacao['Preço_Exec'] = verificacao['Preço_Exec'].astype('float')
verificacao.to_excel(f'O:/SAO/CICH_All\Investment Solutions/12. First Line of Defense/Controles 1LoD/Preco_Medio/Movimentos {datetime.date(dia)}.xlsx')
verificacao['Verificar_Preço'] = abs((verificacao['Preço_Médio_Dia']-verificacao['Preço_Exec']))
verificacao = verificacao.sort_values(by=['Ativo','Corretora','Preço_Exec'])

#Em caso de apontamento da 2º linha de defesa
verificacao['Data'] = dia
verificacao.to_excel('verificacao_pu_medio.xlsx')


#Retirando os titulos publicos e veto power
verificacao = verificacao[~verificacao['Ativo'].str.contains('LFT')]
verificacao = verificacao[~verificacao['Ativo'].str.contains('NTN')]

#Gerando relatótio para trading desk
verificacao = verificacao[verificacao['Verificar_Preço']>0.000001].reset_index(drop=True).drop(columns=['Verificar_Preço'])
verificacao['Verificar']=np.nan
verificacao['Verificar'].loc[verificacao['Veto'].notnull()] = 'ok'
verificacao.to_excel(f'O:/SAO/CICH_All\Investment Solutions/12. First Line of Defense/Controles 1LoD/Preco_Medio/{datetime.date(dia)}.xlsx')
verificacao.to_excel('C:/Temp/Foundation/pu_medio.xlsx')

#envio de e-mail
data_email = "{:%d/%m/%Y}".format(dia)
text_fundo =   '''Prezados .</h3><br>
                 Segue análise de preço médio do dia versus execução.".</h3><br>                                      
                '''        
subject= f'Análise de Preço Médio {data_email} [FLoD]'

to = ['portfolio@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com','marcos.sposito@juliusbaer.com','igor.davanzo@juliusbaer.com','andre.lett@juliusbaer.com']       

if len(boletas)==0:
    text_fundo = 'Não há boletas'
    
email = Email(to = to , subject = subject, text= text_fundo ,send = False,attachments = 'C:/Temp/Foundation/pu_medio.xlsx')      


#Em caso de apontamento da 2º linha de defesa
verificacao['Data'] = dia
verificacao.to_excel('verificacao_pu_medio.xlsx')


In [4]:
def moeda(my_value):
    a = '{:,.2f}'.format(float(my_value))
    b = a.replace(',','v')
    c = b.replace('.',',')
    return c.replace('v','.')

In [5]:
##Controle de movimentações de produtos não recomendados

#Retorna o rating dos produtos da casa
rating_produtos = produtos[['new_productid','new_ratinggpsname']].reset_index()
rating_produtos['productid'] = rating_produtos['productid'].str.lower()

data_fim = fdt.workday(hoje,-1)

#Retorna todas as boletas do sistema e desconsidera aquelas com status diferente de liquidada ou movimentação solicitada
#boletas = bol.boletas_lista_datamov(data_bol,deletado = False, exportado = True)
boletas = bol.boletas_range_datas(data_ini=data_inicial_bol, data_fim=data_fim, exportado=True, deletado=False)
boletas = boletas[boletas['TipoMov']=='C']
boletas=boletas.reset_index()
boletas = boletas[['DataMov','ContaCRM','AtivoGuid', 'AtivoNome','StatusCRM']]
boletas_desconsiderar = [None,'Devolvida','Cancelada','Pré-Boleta']
boletas = boletas.loc[~boletas['StatusCRM'].isin(boletas_desconsiderar)]
boletas['AtivoGuid'] = boletas['AtivoGuid'].str.lower()
boletas['AtivoGuid'] = boletas['AtivoGuid'].apply(lambda x: x.replace('{','').replace('}',''))

#retona o rating dos produtos que sairam da BL Pré Boletas
boletas = pd.merge(left=boletas,right=rating_produtos,left_on='AtivoGuid',right_on='productid',how='left').drop(columns=['productid','new_productid'])
boletas = boletas[(boletas['new_ratinggpsname']!='Recomendado')]
boletas = boletas.drop_duplicates()

#Salvar as boletas do mês
boletas.to_excel(f'O:/SAO/CICH_All\Investment Solutions/12. First Line of Defense/Controles 1LoD/Controle Boletas - Diário/boletas_não_monitoradas {hoje.strftime("%b")}.xlsx')

#Enviar no email apenas as boletas feitas ontem
boletas = boletas[boletas['DataMov']==data_fim]
data_email = "{:%d/%m/%Y}".format(hoje)
boletas.to_excel('C:/Temp/Foundation/boletas_não_monitoradas.xlsx')

#envio de e-mail
text_fundo =   '''Prezados .</h3><br>
                 Segue anexo as boletas feitas ontem em ativos não recomendados ou sem "recomendaçao".</h3><br>                                      
                '''        
subject= f'Monitoramento de boletas em ativos não aprovados {data_email} [FLoD]'

to = ['portfolio@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com']       

if len(boletas)==0:
    text_fundo = 'Não há boletas'
    
email = Email(to = to , subject = subject, text= text_fundo ,send = False,attachments = 'C:/Temp/Foundation/boletas_não_monitoradas.xlsx')      


NameError: name 'produtos' is not defined

In [14]:
###Controler de Insider

##Puxa as classes do databases (consultas nas bases sql da empresa)
crm = Crm()
bds = BDS()
fdt = FuncoesDatas()

## Parametros de data para puxar as boletas do sistema
hoje = bds.banco.hoje()
dia = hoje + relativedelta(months=-1)
mes,ano = hoje.month,hoje.year

#Traz o primeiro dia util do mes anterior
data_inicial_bol = datetime(ano,mes,1)
data_inicial_bol= fdt.verificar_du(data_inicial_bol)


#Traz o ultimo dia do mes
dia_fim_bol = hoje + timedelta(-1)

#Usar só no mensal
data_inicial_bol = datetime(2024,11,21)
dia_fim_bol = datetime(2024,12,9)
mes,ano = data_inicial_bol.month,hoje.year

## Retorna todos os movimentos do mes (proc rv do crm) - PUs médios

movs = crm.movs_rv(data_inicial_bol,dia_fim_bol)
movs = movs[movs['new_statusboleta']==5]
movs = movs[movs['new_databoleta']>data_inicial_bol]

# Retorna apenas as colunas necessarias das movimentações
movs = movs[['new_databoleta','NomeTitular','new_tipomovimentorv','NomeProduto','new_idsistemaorigem','DataLiquidacao','Quantidade', 'PU','ValorOperacao']]

## Trazer o PU médio do dia por cliente e produto
movs['mult'] = movs['PU'] * movs['Quantidade']
movs = movs.groupby(['new_databoleta','NomeTitular','new_idsistemaorigem','new_tipomovimentorv'])[['mult','Quantidade']].sum()
movs['media preco'] = movs['mult'] / movs['Quantidade']
movs = movs.reset_index().drop(columns={'mult','Quantidade'}).replace(1,'Compra').replace(2,'Venda')
movs['new_databoleta'] = movs['new_databoleta'].apply(lambda x: datetime.date(x))
#Salva as movimentações em um excel
movs.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Insider - Variação RV/Archive/Movimentações_Controle_Insider {hoje.strftime("%b")} {hoje.year}.xlsx')

## Retorna o preço médio 5 dias antes e 5 dias depois

# retorna a query de preços
consulta_precos = bds.busca_preço_rv(hoje).sort_values(by=['Ação','Data']).fillna(0).reset_index(drop=True).dropna()
data_precos = data_inicial_bol + timedelta(-10)
consulta_precos = consulta_precos[consulta_precos['Data']>=data_precos]
consulta_precos['Data'] = consulta_precos['Data'].apply(lambda x: datetime.date(x))

#Salva as movimentações em um excel
consulta_precos.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Insider - Variação RV/Archive/Preços_Insider {hoje.strftime("%b")} {hoje.year}.xlsx')

# Traz os valores no formato de linha para fazer a média por coluna (data)
pivot_acao = consulta_precos.groupby(['Data','Ação'])['Preço de Fechamento'].mean().unstack('Ação')
pivot_acao=pivot_acao.dropna(subset=['PETR3'])

#retorna a média móvel 5 dias à frente

pivot_pmedio_posterior=pivot_acao.shift(-4)
pivot_pmedio_posterior = pivot_pmedio_posterior.rolling(5,min_periods=4).mean()

#Retorna num padrão tabela vertical
pivot_pmedio_posterior =pivot_pmedio_posterior.dropna(how= 'all', axis= 0).stack().reset_index().rename(columns={0:'Preço_médio_5+'})

#retorna a média móvel 5 dias à frente
pivot_pmedio_anterior = pivot_acao.rolling(5,min_periods=4).mean()
#Retorna num padrão tabela vertical
pivot_pmedio_anterior =pivot_pmedio_anterior.stack().reset_index().rename(columns={0:'Preço_médio_5-'})

#Unifica com a planilha de trades
preco_medio= pd.merge(left=pivot_pmedio_anterior,right=pivot_pmedio_posterior,on=['Data','Ação'],how='left')

#Unifica com a planilha de trades
insider_control = pd.merge(left=movs,right=preco_medio,left_on=['new_databoleta','new_idsistemaorigem'],right_on=['Data','Ação'],how='left')
insider_control = insider_control.drop(columns=['Data','new_idsistemaorigem']).dropna(subset=['Ação'])

#Incluir as verificações
insider_control['dif_media_-5'] = (insider_control['media preco']/insider_control['Preço_médio_5-'])-1
insider_control['dif_media_+5'] = (insider_control['Preço_médio_5+']/insider_control['media preco'])-1
insider_control['dif_media_-5']= round(insider_control['dif_media_-5']*100,2)
insider_control['dif_media_+5']  = round(insider_control['dif_media_+5']*100,2)
insider_control['verificar_dif_preço'] = insider_control.apply(lambda row : 'ok' if ((abs(row['dif_media_-5']) < 5) & (abs(row['dif_media_+5'])<5)) else 'desenquadrado',axis=1)
insider_control['verificar_insider'] = insider_control.apply(lambda row : 'desenquadrado' if ((row['dif_media_+5']>5) & (row['new_tipomovimentorv']=='Compra')) | ((row['dif_media_+5'] < -5) & (row['new_tipomovimentorv']=='Venda')) else 'ok',axis=1)


#Salvando o controle de insider do mês
insider_control.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Insider - Variação RV/Archive/controle de insider {hoje.strftime("%b")} {hoje.year}.xlsx')

#Envio do e-mail 

verificar = insider_control[['new_databoleta', 'new_tipomovimentorv', 'Ação', 'media preco',
       'Preço_médio_5-', 'Preço_médio_5+', 'dif_media_-5', 'dif_media_+5','verificar_dif_preço',
       'verificar_insider']].rename(columns={'new_databoleta':'Data_Boleta','new_tipomovimentorv':'Tipo_Movimento'})

verificar = verificar[(verificar['verificar_insider']!='ok')&(verificar['verificar_dif_preço']!='ok')]
verificar = verificar[(verificar['Data_Boleta'] <= datetime.date(fdt.workday(fdt.hoje(),-6)))]

tbl1 = build_table(verificar,'blue_dark',text_align = 'center',width="300px")
data_email = "{:%d/%m/%Y}".format(hoje)
text_fundo =   f'''Prezados .</h3><br>
                 Seguem os apontamento de diferenças de preços com a média dps 5 dias anteriores e posteriores.</h3><br>
                {tbl1}.</h3><br>                                          
                '''        
subject= f'Monitoramento de operações de RV {data_email} [FLoD]'
to = ['portfolio@juliusbaer.com','marcos.sposito@juliusbaer.com','igor.davanzo@juliusbaer.com','andre.lett@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com']            
if len(verificar)==0:
    text_fundo = 'Não há apontamentos'
email = Email(to = to , subject = subject, text= text_fundo ,send = False)      


In [7]:
###Controle de compromissada

posicao = PosicaoDm1()

#Retorna todos os fundos qualificados pelo cadastro de produto CRM
fundos_qualificados = crm_fundos.fundo_cadastro()

#Filtra apenas produtos gestão JBFO

id_gestao = ['5A865D62-45FE-DF11-BAE8-D8D385B9752E','85475FA5-E062-EA11-8E83-005056912B96','2DA233C3-44FE-DF11-BAE8-D8D385B9752E']
fundos_qualificados = fundos_qualificados.loc[fundos_qualificados['new_gestorid'].isin(id_gestao)]
fundos_qualificados = fundos_qualificados[fundos_qualificados['new_publicoalvoname']=='Qualificado'][['new_idsistemaorigem']]

#Pegar o código do produto  para trazer a posição da po_carteira

cadastro= posicao.po_cadastro()[['GuidContaCRM','NomeContaCRM','CodigoProduto']]
cadastro = pd.merge(left =fundos_qualificados, right =cadastro,left_on='new_idsistemaorigem',right_on ='CodigoProduto', how='left')[['GuidContaCRM']].dropna()
cadastro['GuidContaCRM'] = cadastro['GuidContaCRM'].str.lower()

#Retorna a posicao dos fundos
posicao_fundos = posicao.posicao_fundos_all(hoje, apenas_exclusivos=False)
posicao_fundos['GuidContaCRM'] = posicao_fundos['GuidContaCRM'].str.lower()
posicao_fundos = pd.merge(left = posicao_fundos, right =cadastro,on='GuidContaCRM', how='right')

#Agrupa todos os fundos por produtos (produto tipo cotas de fundos) e verifica se está com mais de 10 % - Fundos de Liquidez
verificar_cotas_liquidez = posicao_fundos.groupby(['NomeContaCRM','TipoProduto','GuidProduto','NomeProduto', 'Classe'])[['PesoFinal']].sum().reset_index()
verificar_cotas_liquidez['PesoFinal'] = verificar_cotas_liquidez['PesoFinal']*100
fundos_liquidez=['838559d4-3f1f-ee11-8d7a-005056b17af5','5aa8e514-0fc2-dd11-9886-000f1f6a9c1c','f4a7e952-713a-e311-87ef-000c29cb7e20','8ba7e514-0fc2-dd11-9886-000f1f6a9c1c','fee2528e-541d-ec11-a569-005056912b96','4a2ee0b8-2a93-e311-a874-000c29cb7e20']
verificar_cotas_liquidez = verificar_cotas_liquidez.loc[verificar_cotas_liquidez['GuidProduto'].isin(fundos_liquidez)]
verificar_cotas_liquidez = verificar_cotas_liquidez[~verificar_cotas_liquidez['NomeContaCRM'].str.contains('FIC')]
verificar_cotas_liquidez = verificar_cotas_liquidez[verificar_cotas_liquidez['PesoFinal']>7]

#Envia e-mail para PM
tbl1 = build_table(verificar_cotas_liquidez,'blue_dark',text_align = 'center',width="300px")
data_email = "{:%d/%m/%Y}".format(hoje)
text_fundo =   f'''Prezados .</h3><br>
                 Favor verificar as posições abaixo pois, podem gerar desenquadramentos em fundos qualificados.</h3><br>
                {tbl1}.</h3><br>                                          
                '''        
subject= f'Fundos Qualificados - Exposição em fundos DI {data_email} [FLoD]'
to = ['portfolio@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com']            
if len(verificar_cotas_liquidez)==0:
    text_fundo = 'Não há apontamentos'
email = Email(to = to , subject = subject, text= text_fundo ,send = False)      


In [8]:
##Controle de compromissadas em fundos de previdencia e fundos de cotas

posicao = PosicaoDm1()

#Retorna todas as posições dos fundos que possuem compromissada
posicao_fundos = posicao.posicao_fundos_all(hoje, apenas_exclusivos=False)
posicao_compromissada = posicao_fundos[posicao_fundos['TipoProduto']=='COMP']

#retorna todos os fundos FICS e Previdenciarios
posicao_compromissada_fic = posicao_compromissada[posicao_compromissada['NomeContaCRM'].str.contains('FIC')][['NomeContaCRM','FinanceiroFinal']]
posicao_compromissada_prev = posicao_compromissada[posicao_compromissada['NomeContaCRM'].str.contains('Prev')][['NomeContaCRM','FinanceiroFinal']]

#Retorna os PLs
pl = posicao.fundos_pl_estimado()[['NomeContaCRM','PL']]

#faz o merge das posições com os PLs dos fundos
posicao_compromissada_fic = pd.merge(left=posicao_compromissada_fic,right=pl, on='NomeContaCRM',how='left')
posicao_compromissada_prev = pd.merge(left=posicao_compromissada_prev,right=pl, on='NomeContaCRM',how='left')

#Gera o calculo da posição
posicao_compromissada_fic['Peso'] = posicao_compromissada_fic['FinanceiroFinal'] / posicao_compromissada_fic['PL']*100
posicao_compromissada_fic['Alerta'] = 'Compromissada acima de 3%'
posicao_compromissada_fic = posicao_compromissada_fic[posicao_compromissada_fic['Peso']>2.5]

#Filtrando apenas os masters dos fundos de previdencia
posicao_compromissada_prev['Peso'] = posicao_compromissada_prev['FinanceiroFinal'] / posicao_compromissada_prev['PL']*100
posicao_compromissada_prev = posicao_compromissada_prev[~posicao_compromissada_prev['NomeContaCRM'].str.contains('PGBL')]
posicao_compromissada_prev = posicao_compromissada_prev[~posicao_compromissada_prev['NomeContaCRM'].str.contains('VGBL')]
posicao_compromissada_prev['Alerta'] = 'Compromissada acima de 10%'
posicao_compromissada_prev = posicao_compromissada_prev[posicao_compromissada_prev['Peso']>8]

#Verificando os fundos desenquadrados
alertas = pd.concat([posicao_compromissada_fic,posicao_compromissada_prev])

#Envia e-mail para PM sobre as posições de compromissada
tbl1 = build_table(alertas,'blue_dark',text_align = 'center',width="300px")
data_email = "{:%d/%m/%Y}".format(hoje)
text_fundo =   f'''Prezados .</h3><br>
                 Seguem posições relevantes em operações compromissadas.</h3><br>
                {tbl1}.</h3><br>                                          
                '''        
subject= f'Posição em operações compromissadas - Atenção {data_email} [FLoD]'
to = ['portfolio@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com']            
if len(alertas)==0:
    text_fundo = 'Não há apontamentos'
email = Email(to = to , subject = subject, text= text_fundo ,send = False)  


In [15]:
import requests
import json


posicao = PosicaoDm1()
dia = fdt.hoje()
dia = fdt.workday(hoje, -3)



rv = bds.consulta_ativos_RV()
rv['productid'] = rv['productid'].str.lower()
rv = rv[['productid','Nome']]

#Com FII
# rv = bawm.ativos_po_ativos_rv()
# rv['GuidAtivo'] = rv['GuidAtivo'].str.lower()
# rv = rv[['GuidAtivo','CodNegociacao']]

url = "https://svrprod002.gps.br/bds/api/HistoricoNegociacao/BuscaHistorico/2024-10-29/2024-11-29"
page = requests.get(url, verify=False)
data = page.json()
data_str = json.dumps(data)
data_dict = json.loads(data_str)
acoes = pd.DataFrame(data_dict)
acoes['dataFechamento']  = pd.to_datetime(acoes['dataFechamento'])
acoes = acoes[acoes['dataFechamento']==dia]

acoes['ValorNegociado'] = acoes['qtdeAcoesEmitidas'] * acoes['fechamento']
acoes = acoes[['ticker','qtdeAcoesEmitidas','ValorNegociado']].drop_duplicates()


posicao = posicao.posicao_fundos_all(dia, apenas_exclusivos=False)

posicao = pd.merge(left=rv,right=posicao,left_on='productid',right_on='GuidProduto',how='left') 
posicao=posicao.dropna(subset=['GuidProduto'])
posicao = posicao.groupby(['Nome']).sum().reset_index()[['Nome','FinanceiroFinal','QuantidadeFinal']].drop_duplicates()

posicao = pd.merge(left=posicao, right=acoes,left_on='Nome',right_on='ticker',how='left')
posicao = posicao[['Nome','FinanceiroFinal','QuantidadeFinal','qtdeAcoesEmitidas','ValorNegociado']].drop_duplicates()
posicao['%FInanceiro'] = round(posicao['FinanceiroFinal'] / posicao['ValorNegociado']*100,2)
posicao['%Quantidade'] = round(posicao['QuantidadeFinal'] / posicao['qtdeAcoesEmitidas']*100,2)   
posicao = posicao.sort_values(by=['%FInanceiro'],ascending=False)
posicao.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/CVM 44 - Participação Relevante/{hoje.strftime("%b")}{hoje.year}.xlsx')



In [16]:
#Controle de day-trade

#Retorna o dia anterior
dia = fdt.workday(hoje, n_dias=-2)
boletas = bol.boletas_importadas_cockpit(dia)

#Cruzar os dados
compras = boletas[boletas['TipoMov']=='C'][['ContaCRMGuid','ContaCRM','AtivoGuid','AtivoNome','Quantidade','Preco','Financeiro']].rename(columns={'Quantidade':'Quantidade_Compra','Financeiro':'Financeiro_Compra','Preco':'Preco_Compra'})
vendas = boletas[boletas['TipoMov']=='V'][['ContaCRMGuid','ContaCRM','AtivoGuid','AtivoNome','Quantidade','Preco','Financeiro']].rename(columns={'Quantidade':'Quantidade_Venda','Financeiro':'Financeiro_Venda','Preco':'Preco_Venda'})
verificar = pd.merge(left=compras,right=vendas, on=['ContaCRMGuid','AtivoGuid'],how='inner')
verificar = verificar [['ContaCRM_x','AtivoNome_x','Quantidade_Compra','Preco_Compra','Financeiro_Compra','Quantidade_Venda','Preco_Venda','Financeiro_Venda']]
verificar = verificar[~verificar['AtivoNome_x'].str.contains('LFT')]
if len(verificar) >=1:
    verificar.to_excel(f'O:/SAO/CICH_All/Investment Solutions/12. First Line of Defense/Controles 1LoD/Controle Boletas - Diário/DayTrade{datetime.date(dia)}.xlsx')
    tbl1 = build_table(verificar,'blue_dark',text_align = 'center',width="300px")
    data_email = "{:%d/%m/%Y}".format(dia)
    text_fundo =   f'''Prezados .</h3><br>
                     Seguem os apontamento de possíveis operações day-trade, favor nos retornar com a justificativa.</h3><br>
                    {tbl1}.</h3><br>                                          
                    '''        
    subject= f'Monitoramento de day trade {data_email} [FLoD]'
    to = ['portfolio@juliusbaer.com','marcos.sposito@juliusbaer.com','igor.davanzo@juliusbaer.com','andre.lett@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com']            
#     if len(verificar)==0:
#         text_fundo = 'Não há apontamentos'
    email = Email(to = to , subject = subject, text= text_fundo ,send = False)        

In [12]:
##Verificar o emprestimos de ativos

#Roda o arquivo do Imbarq

# from sys_Imbarq import imbarq
# imbarq()


#Retornar o arquivo de emprestimo de ativos e faz os ajustes necessarios

def busca_especificacao(nome_especificacao):
    arquivo = 'O:/SAO/CICH_All/Portfolio Management/18-Imbarq/Imbarque_especificacao.xlsx'
    df = pd.read_excel(arquivo)
    df = df[df['Arquivo']==nome_especificacao].sort_values('Ordem')
    return list(df['Lenght'].to_list())

def busca_coluna(nome_especificacao):
    arquivo = 'O:/SAO/CICH_All/Portfolio Management/18-Imbarq/Imbarque_especificacao.xlsx'
    df = pd.read_excel(arquivo)
    df = df[df['Arquivo']==nome_especificacao].sort_values('Ordem')
    return list(df['Coluna'].to_list())

larguras_065265 = busca_especificacao('EmprestimoAtivos')
larguras_065265 = [int(x) for x in larguras_065265]

controle_emprestimo_ativos = pd.read_fwf('O:/SAO\CICH_All/Portfolio Management/18-Imbarq/065265.txt', header=None,widths = larguras_065265)
controle_emprestimo_ativos.columns = busca_coluna('EmprestimoAtivos')

#Ajustes das contas (considerar o digito)
contas = crm.cod_imbarq()
contas= contas.dropna(subset = ['new_numeroconta'])
contas = contas[contas['new_numeroconta'] != '328196-4']
contas['new_numeroconta'] = contas['new_numeroconta'].astype(str)
contas['accountid']=contas['accountid'].str.lower()
contas['new_digitoconta'] = contas['new_digitoconta'].fillna('sem_dig')
contas['new_numeroconta'] = contas['new_numeroconta'].str.lower()
contas['new_digitoconta']= contas['new_digitoconta'].str.lower()
contas['conta'] = contas['new_numeroconta'] + contas['new_digitoconta']
contas['conta'] = contas['conta'].apply(lambda x: x.replace('sem_dig',''))
contas['conta'] = contas['conta'].astype('int')

#Trazer a especificação do cliente ao ativo dado como emprestimo
emprestimo_analisar = pd.merge(left=controle_emprestimo_ativos,right=contas,left_on='Código do Investidor Solicitado',right_on='conta',how='left')
emprestimo_analisar['Quantidade em Liquidação'] = emprestimo_analisar['Quantidade em Liquidação']/1000
emprestimo_analisar['Quantidade Coberta'] = emprestimo_analisar['Quantidade Coberta']/1000


#Colunas necessarias para o e-mail
emprestimo_analisar = emprestimo_analisar[['new_bancoidname','new_titularidadeidname','Data de Movimento','Código de Negociação','Data de negociação','Data de Vencimento','Data de Carência','Quantidade em Liquidação']].rename(columns={'new_titularidadeidname':'Cliente','new_bancoidname':'Custódia'})

#Envia e-mail para PM sobre as operações de emprestimos
tbl1 = build_table(emprestimo_analisar,'blue_dark',text_align = 'center',width="300px")
data_email = "{:%d/%m/%Y}".format(hoje)
text_fundo =   f'''Prezados .</h3><br>
                 O Imbarq está retornando as posições abaixo como BTCs (tomados ou doados).</h3><br>
                {tbl1}.</h3><br>                                          
                '''        
subject= f'Operações em BTC (URGENTE) {data_email} [FLoD]'
to = ['portfolio@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','jbbr_im_risk@juliusbaer.com','liquidacao@juliusbaer.com','luiz.amaral@juliusbaer.com']            
if len(emprestimo_analisar)==0:
    text_fundo = 'Não há apontamentos'
email = Email(to = to , subject = subject, text= text_fundo ,send = False)  


EmptyDataError: No columns to parse from file

In [ ]:
#Calculo das taxas médias - futuro controle

#retorna os fundos gestão JBFO
id_gestao = ['5A865D62-45FE-DF11-BAE8-D8D385B9752E','85475FA5-E062-EA11-8E83-005056912B96','2DA233C3-44FE-DF11-BAE8-D8D385B9752E']
fundos_jbfo  = fundos .loc[fundos['new_gestorid'].isin(id_gestao)]

#retorna a taxa de adm como produtos
fundos_jbfo  = fundos_jbfo[['productid','new_taxadeadministracao']]
fundos_jbfo['productid']=fundos_jbfo['productid'].str.lower()

#Retorna as carteiras dos exclusivos
posicao_fundos = posicao.posicao_fundos_all(hoje, apenas_exclusivos=True)
posicao_fundos['GuidProduto']=posicao_fundos['GuidProduto'].str.lower()

#Traz apenas os produtos de gestão JBFO

taxa_maxima = pd.merge(left=fundos_jbfo,right=posicao_fundos,left_on='productid',right_on='GuidProduto',how='right').rename(columns={'new_taxadeadministracao':'Taxa_ADM'})

#Retira os fundos PGBL e VGBL
taxa_maxima = taxa_maxima[~taxa_maxima['NomeContaCRM'].str.contains('PGBL')]
taxa_maxima = taxa_maxima[~taxa_maxima['NomeContaCRM'].str.contains('VGBL')]

#Ponderar as taxas dos fundos JBFO
taxa_maxima['Taxa_ADM'] = taxa_maxima['Taxa_ADM'].fillna(0)
taxa_maxima['Ponderada'] = taxa_maxima['Taxa_ADM'] * taxa_maxima['PesoFinal']
taxa_maxima=taxa_maxima.groupby(['NomeContaCRM']).sum().reset_index()[['NomeContaCRM','Ponderada']]
taxa_maxima = taxa_maxima.sort_values(by=['Ponderada'],ascending=False).reset_index(drop=True)

#Pegando os 15 primeiros
taxa_maxima = taxa_maxima.head(15)


In [ ]:
#Exposição bancaria
posicao = pd.read_excel('O:/SAO/CICH_All/Portfolio Management/06 - Relatório/Tableau/posdia_Explosao.xlsx')
posicao = posicao[posicao['Distrato']==False]


pl_total = posicao['FinanceiroFinal'].sum()
produtos_bancos = ['CDB','LCI','LCA','LIG']


bancarios = posicao[['Segmento','Perfil','NomeContaCRM','Classe','TipoProduto','NomeProduto','NomeEmissor','PLFinal','FinanceiroFinal']]
bancarios['NomeEmissor'] = bancarios['NomeEmissor'].str.lower()
bancarios = bancarios.loc[bancarios['TipoProduto'].isin(produtos_bancos)].rename(columns={'NomeEmissor':'Banco'})

bancos_aprovados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Bancos JBFO')
bancos_aprovados['Banco'] = bancos_aprovados['Banco'].str.lower()

posicao = posicao[posicao['Distrato']==False]
pl_total = posicao['FinanceiroFinal'].sum()
bancarios = bancarios[['Segmento','Perfil','NomeContaCRM','Classe','TipoProduto','NomeProduto','Banco','PLFinal','FinanceiroFinal']]

bancarios = bancarios.loc[bancarios['TipoProduto'].isin(produtos_bancos)].rename(columns={'NomeEmissor':'Banco'})
bancarios = pd.merge(left=bancarios, right=bancos_aprovados, on='Banco',how='left')

#Apenas ativos bancários
pl_total_bancos = bancarios['FinanceiroFinal'].sum()


In [ ]:
#Apenas ativos bancários
pl_total_bancos = bancarios['FinanceiroFinal'].sum()
bancarios['Status'] = bancarios['Status'].fillna('Não Aprovado')
bancarios['Banco'] = bancarios['Banco'].astype('str')
bancarios['Banco'] = bancarios['Banco'].apply(lambda x : x.title())
bancarios['Aprovacao'] = bancarios['Status']

#Exposição por banco
bancarios_total = bancarios.groupby(['Banco','Aprovacao']).sum()[['FinanceiroFinal']].reset_index().sort_values(by=['FinanceiroFinal'],ascending=False)
bancarios_total = bancarios_total.reset_index(drop=True)
bancarios_total['%'] = bancarios_total['FinanceiroFinal'] / pl_total_bancos * 100
bancarios_total['%'] = round(bancarios_total['FinanceiroFinal'] / pl_total_bancos * 100,2)
bancarios_total['FinanceiroFinal']=bancarios_total['FinanceiroFinal'].apply(lambda x : moeda(round(x)))

#Exposição por tipo de ativo bancário
bancarios_tipo_ativo = bancarios.groupby(['TipoProduto','Aprovacao']).sum()[['FinanceiroFinal']].reset_index().sort_values(by=['Aprovacao','FinanceiroFinal'],ascending=[True,False])
bancarios_tipo_ativo = bancarios_tipo_ativo.reset_index(drop=True)
bancarios_tipo_ativo['%'] = bancarios_tipo_ativo['FinanceiroFinal'] / pl_total_bancos * 100
bancarios_tipo_ativo['%'] = round(bancarios_tipo_ativo['FinanceiroFinal'] / pl_total_bancos * 100,2)
bancarios_tipo_ativo['FinanceiroFinal']=bancarios_tipo_ativo['FinanceiroFinal'].apply(lambda x : moeda(round(x)))

#Dos Bancarios quantos são monitorados ou aprodos

bancarios_aprovacao = bancarios.groupby(['Aprovacao']).sum()[['FinanceiroFinal']].reset_index().sort_values(by=['Aprovacao','FinanceiroFinal'],ascending=[True,False])
bancarios_aprovacao = bancarios_aprovacao.reset_index(drop=True)
bancarios_aprovacao['%'] = round(bancarios_aprovacao['FinanceiroFinal'] / pl_total_bancos * 100,2)
bancarios_aprovacao['FinanceiroFinal']=bancarios_aprovacao['FinanceiroFinal'].apply(lambda x : moeda(round(x)))

#Dos Bancarios quantos são monitorados ou aprovados por segmento

bancarios_aprovacao_segmento = bancarios.groupby(['Segmento','Aprovacao']).sum()[['FinanceiroFinal']].reset_index().sort_values(by=['Aprovacao','FinanceiroFinal'],ascending=[True,False])
bancarios_aprovacao_segmento = bancarios_aprovacao_segmento.reset_index(drop=True)
bancarios_aprovacao_segmento['%'] = round(bancarios_aprovacao_segmento['FinanceiroFinal'] / pl_total_bancos * 100,2)
bancarios_aprovacao_segmento['FinanceiroFinal']=bancarios_aprovacao_segmento['FinanceiroFinal'].apply(lambda x : moeda(round(x)))

#Email dos Bancarios
tbl1 = build_table(bancarios_total,'blue_dark',text_align = 'center',width="300px")
tbl2 = build_table(bancarios_tipo_ativo,'blue_dark',text_align = 'center',width="300px")
tbl3 = build_table(bancarios_aprovacao,'blue_dark',text_align = 'center',width="300px")
tbl4 = build_table(bancarios_aprovacao_segmento,'blue_dark',text_align = 'center',width="300px")


hoje = "{:%d/%m/%Y}".format(fdt.hoje())
text_fundo =   f'''Prezados .</h3><br>
                 Seguem números dos Bancos Aprovados x Não Aprovados de nossa base de ativos.</h3><br>
                {tbl3}.</h3><br> 
                 Seguem números por bancos e sua recomendação.</h3><br>
                {tbl1}.</h3><br> 
                Seguem números por tipo de produto de emissão bancária e sua recomendação.</h3><br>
                {tbl2}.</h3><br> 
                Seguem números por segmentação comercial</h3><br>
                {tbl4}.</h3><br> 
                
                
                '''        
subject= f'Segue Análise de exposição em ativos emitidos por bancos {hoje} [FLoD]'
to = ['gabriela.brotto@juliusbaer.com','mariana.drumond@juliusbaer.com','tamara.alves@juliusbaer.com','luiza.gomes@juliusbaer.com']            

email = Email(to = to , subject = subject, text= text_fundo ,send = False) 

In [ ]:
def identifica_mandato(nome,tipo):
    if 'infra' in nome and 'exclusivo' in perfil:
        tipo = 'Infra_Exclusivp'